In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

training_v2 = pd.read_csv("/content/drive/My Drive/Datathon_2020/widsdatathon2020/training_v2.csv")
data_dict = pd.read_csv("/content/drive/My Drive/Datathon_2020/widsdatathon2020/WiDS Datathon 2020 Dictionary.csv")
# Any results you write to the current directory are saved as output.

In [ ]:
# Cleaning and preparing the data
def clean_and_prep(df, features, scaler):
  df_x = df[features].copy()
  categorical_cols = df_x.columns[df_x.dtypes==object].tolist()
  list_of_num_features = df_x.columns[df_x.dtypes != object].tolist()
  # print(df_x.shape)
  # removing outliers
  sigma = 2
  df_x[list_of_num_features] = df_x[list_of_num_features].mask(df_x[list_of_num_features].sub(df_x[list_of_num_features].mean()).div(df_x[list_of_num_features].std()).abs().gt(sigma))
  # print(df_x.shape)
  # fixing NaN values. Mean for numeric and most common for categorical
  #for categorical. Pick the most common value
  for i in categorical_cols:
    df_x[i] = df_x[i].fillna(df_x[i].value_counts().index[0])
  # print(df_x.shape)
  #for numeric !!!!probably need to fix!!!!! Replacing with mean value.
  df_x.replace({-1 : np.NaN}, inplace=True) #replacing -1 for apache prob
  # print(df_x.shape)
  df_x[list_of_num_features] = df_x[list_of_num_features].fillna(df_x[list_of_num_features].mean())
  # print(df_x.shape)
  #replacing categorical features with "one hot encoding"
  df_x = pd.get_dummies(df_x)
  # print(df_x.shape)
  #get the new feature list
  new_features = [i.replace(' ','_').replace('/','_') for i in df_x.columns.tolist()]
  
  #scale features
  # scaler = StandardScaler()
  df_x = scaler.fit_transform(df_x)
  df_x = pd.DataFrame(df_x, columns=new_features)
  # print(df_x.shape)
  return df_x

In [ ]:
# Remove Features with more than 80 percent missing values
ratio =  0.80
y = training_v2['hospital_death']
train = training_v2.copy()
train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
train_missing = train_missing.index[train_missing > ratio].tolist()

train = train.drop(train_missing, axis=1)
train = train.drop('readmission_status', axis=1)


In [ ]:
#Split to train and test sets
from sklearn.model_selection import train_test_split

features = train.columns.tolist()[4:] #here I exclude columns we are not using as features including hospital_death

X_train, X_test, y_train, y_test = train_test_split(train[features], y, test_size=0.25)

# Cleaning train and test sets
X_train = clean_and_prep(X_train, features, StandardScaler())
X_test = clean_and_prep(X_test, features, StandardScaler())
#make sure you have same number of features for both test and train sets/ In my case 199
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(68784, 199) (68784,)
(22929, 199) (22929,)


In [ ]:
# Downsampling 
from imblearn.under_sampling import RandomUnderSampler
sampling_strategy = 0.25
sampler = RandomUnderSampler(random_state=42, sampling_strategy=sampling_strategy)

features = X_train.columns.tolist()

X_train_und, y_train_und = sampler.fit_resample(X_train, y_train)
X_train_und = pd.DataFrame(X_train_und, columns=features)
y_train_und = pd.Series(y_train_und)
print(X_train_und.shape, y_train_und.shape)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


(29735, 199) (29735,)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Upsampling
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)

features = X_train_und.columns.tolist()

X_train_ups, y_train_ups = sm.fit_resample(X_train, y_train)

X_train_ups = pd.DataFrame(X_train_ups, columns=features)
y_train_ups = pd.Series(y_train_ups)
print(X_train_ups.shape, y_train_ups.shape)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(125674, 199) (125674,)


In [ ]:
# Running the models:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Function for running a model
def run_model(name, model, X_train, X_test, y_train, y_test):
  model.fit(X_train, y_train)

  y_test_pred = model.predict_proba(X_test)
  m_auc = roc_auc_score(y_test, y_test_pred[:, 1])
  # summarize scores
  print('{0}: ROC AUC={1:1.4f}'.format(name,m_auc))




In [ ]:
# Here I run different models with full data, upsampled and downsampled for comparison
# you can use any sckitlearn model with the function
from xgboost import XGBClassifier
params_xgb = {'random_state': 42, 'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 7}

xgb = XGBClassifier(**params_xgb)
lr = LogisticRegression(C=0.01, solver='liblinear', random_state=42)

#fill data
run_model("LogisticRegression", lr, X_train, X_test, y_train, y_test)
run_model("XGB", xgb, X_train, X_test, y_train, y_test)

# undersample
run_model("LogisticRegression undersample", lr, X_train_und, X_test, y_train_und, y_test)
run_model("XGB undersample", xgb, X_train_und, X_test, y_train_und, y_test)

# undersample
run_model("LogisticRegression upsample", lr, X_train_ups, X_test, y_train_ups, y_test)
run_model("XGB upsample", xgb, X_train_ups, X_test, y_train_ups, y_test)

LogisticRegression: ROC AUC=0.8613
XGB: ROC AUC=0.8811
LogisticRegression undersample: ROC AUC=0.8622
XGB undersample: ROC AUC=0.8814
LogisticRegression upsample: ROC AUC=0.8548
XGB upsample: ROC AUC=0.8046


In [ ]:
#Plot feature importancies for the model. 
importances = xgb.feature_importances_
imp_features = feat_importances.nlargest(20).index.tolist()
feat_importances = pd.Series(importances, index=X_train_und.columns)
feat_importances.nlargest(100).plot(kind='barh')

In [ ]:
# # features with low variance Under the testing havent finished yet
# low_var_cols = {}
# for i in list_of_num_features:
#     if train[i].value_counts().count() < 4:
#         low_var_cols[i] = train[i].value_counts().count()


In [ ]:
#parameters tuning for LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
parameters = {
#     "penalty": ["l1", "l2"],
    "C": [0.01, 0.05, 0.1, 0.2, 0.5, 0.8],
    "solver":["liblinear","lbfgs"]
    }
clf = GridSearchCV(LogisticRegression(random_state=0), parameters, cv=3, n_jobs=-1, scoring='roc_auc').fit(X_train_ups, y_train_ups)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

In [ ]:
#parameters tuning for XGBClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
parameters = {"learning_rate"    : [0.05, 0.15, 0.25, 0.30 ] ,
              "max_depth"        : [ 3, 5, 8, 10],
              "min_child_weight" : [ 1, 3, 7 ],
              "gamma"            : [ 0.0, 0.1, 0.3 ],
              "colsample_bytree" : [ 0.3, 0.5 , 0.7 ] }
clf = GridSearchCV(XGBClassifier(random_state=0), parameters, cv=3, n_jobs=-1, scoring='roc_auc').fit(X_train_und, y_train_und)

print("Best parameters set found on development set:")
print()
res_file = open("/content/drive/My Drive/Datathon_2020/best_params.txt","w") 
print(clf.best_params_, file = res_file)
res_file.close()
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
res_file = open("/content/drive/My Drive/Datathon_2020/grid_res.txt","w") 
 
print(classification_report(y_true, y_pred), file = res_file)
res_file.close()

Best parameters set found on development set:


Grid scores on development set:

0.880 (+/-0.006) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1}
0.880 (+/-0.006) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3}
0.880 (+/-0.006) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 7}
0.888 (+/-0.005) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1}
0.889 (+/-0.006) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 3}
0.889 (+/-0.005) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 7}
0.892 (+/-0.004) for {'colsample_bytree': 0.3, 'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 1}
0.892 (+/-0.004) for {'colsample_bytree': 0.3, 'gamma': 0.